<a href="https://colab.research.google.com/github/shubham6907/Python-Codes/blob/main/Postgres_Replication_and_FDW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Postgres Replication**

In [ ]:
# Publication
CREATE PUBLICATION sample_publication
    FOR TABLE public.sample
    WITH (publish = 'insert, update, delete, truncate', publish_via_partition_root = false);

create table table1(x int primary key, y int);

insert into table1 values(10, 11);

create publication my_publication for table table1;

insert into table1 values(30, 31);

select * from table1;


# Subscription
CREATE SUBSCRIPTION mysub
       CONNECTION <conn stuff>
       PUBLICATION foo
       WITH (slot_name=my_slot, create_slot=false);


create publication my_publication for table table1;
CREATE SUBSCRIPTION my_subscription CONNECTION 'host=localhost port=5432 dbname=postgres' PUBLICATION my_publication WITH (connect =true, slot_name=my_sub_slot, create_slot=true, enabled=true);

select * from table1;

delete from table1;


# Slot creation
select pg_create_logical_replication_slot('my_sub_slot','pgoutput');
select pg_logical_slot_get_binary_changes('my_sub_slot', NULL, NULL, 'publication_names', 'my_publication', 'proto_version', '1');
CREATE SUBSCRIPTION my_subscription CONNECTION 'host=localhost port=5432 dbname=postgres' PUBLICATION my_publication WITH (connect =true, slot_name=my_sub_slot, create_slot=false, enabled=true);

## **Postgres Foreign Data Wrappers (FDW)**

In [ ]:
CREATE EXTENSION postgres_fdw;

DROP SERVER IF EXISTS abc_server CASCADE;
CREATE SERVER abc_server
    FOREIGN DATA WRAPPER postgres_fdw
    OPTIONS (host '$PG_EDGE_HOST$', dbname '$MAR_DBname$', port '$PG_EDGE_PORT$');

DROP USER MAPPING IF EXISTS FOR current_user SERVER abc_server;

CREATE USER MAPPING FOR current_user
SERVER abc_server
OPTIONS (user '$MAR_DBuser$', password '$MAR_DBpwd$');

IMPORT FOREIGN SCHEMA public LIMIT TO (table1, table2)
   FROM SERVER abc_server INTO public;

DROP TABLE IF EXISTS view1_table CASCADE;

CREATE OR REPLACE VIEW view1_table AS
SELECT
*
from table1 JOIN table2 ON table1.column = table2.column;


# Concatenation
table.column1::text||' '||table.column2||' '||table.column3::text as column4

# Coalesce
COALESCE ((SELECT temp.temp_desc FROM temp_data temp WHERE
        temp.temp_recipe like '%' || TRIM(table1.srecipegroup) || '%'
        AND temp.temp_min_quantity <= table1.fquantity and table1.fquantity < temp.temp_max_quantity
        AND temp.temp_application like '%application%' LIMIT 1), 'unknown') column5

# Trim
trim(split_part(temp_reason_for_exc, '(', 1)) as temp_reason

# Round
round((extract(epoch from exception_end_time-exception_start_time)/3600)::numeric, 2) as temp_exception_duration